In [1]:
using Revise

using MFAScan.Ensemble
using MFA.Ensemble
using Random
using LatticeModels
using Lattices
using LinearAlgebra
using ProfileView
using KrylovKit

[ Info: Precompiling MFAScan [top-level]


In [2]:
struct CubicParams
    L::Int
    W::Float64
end

In [3]:
function f(p::CubicParams; rng = Random.GLOBAL_RNG)
    H = cubic(L = p.L, M = p.L, N = p.L) .+ p.W*Diagonal(rand(L*L*L) .- 0.5)
end

function g(p::CubicParams; rng = Random.GLOBAL_RNG)
    H = square(M = p.L, N = p.L) .+ p.W*Diagonal(rand(L*L) .- 0.5)
end

function h(p::CubicParams; rng = Random.GLOBAL_RNG) #Aubry Andre
    H = chain(N = p.L) .+ p.W*Diagonal(cos.(2π*(1:L)*(1 + √5)/2 .+ 2*pi*rand(rng)))
end

h (generic function with 1 method)

In [42]:
L = 10; W = 5.0; E_c = 0.0; E_del = 0.1
l = [1]
q = [2.0]

p = CubicParams(L, W)
ltc = Lattice3D(L, L, L, 1)
p_MFA = MFAParameters(ltc, l, q)
prepare_MFA!(p_MFA)

In [46]:
using MFAScan.Ensemble

@time a, b, c, _= mt_scan_ταf(f,
    p, 
    E_c, 
    E_del, 
    p_MFA,
    R=100, 
    c = 2000.0,
    nev = 10, 
    rng = MersenneTwister(1234))

  0.893317 seconds (162.47 k allocations: 700.040 MiB, 1.07% gc time)


(-0.0002973246783800491, [2.468121068475237;;], [0.01128867724580137;;], [2.3153516525839812;;], [0.018629163195522533;;], [2.1625822366927254;;], [0.026367637550576833;;])

In [41]:
b

1×1 Matrix{Float64}:
 169.14882792429316

In [81]:
@profview a, b, c, _= mt_scan_ταf(f,
    p, 
    E_c, 
    E_del, 
    p_MFA,
    R=20, 
    c = 1.0,
    nev = 10, 
    rng = MersenneTwister(1234))

Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=2, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile  -  01:46:11", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip